In [37]:
import spotipy
import spotipy.util as util
import pandas as pd
import os
import time
import requests
from bs4 import BeautifulSoup as bs
import datetime
from datetime import timedelta, date
import glob
from os import listdir
import numpy as np
import pymongo
import csv
import json
import sys, getopt, pprint
from pymongo import MongoClient
import types
import itertools
from itertools import zip_longest

import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [38]:
def current_user_recently_played(self, limit=50):
    return self._get('me/player/recently-played', limit=limit)

In [39]:
scope="user-read-recently-played user-read-private user-top-read user-read-currently-playing"

In [40]:
if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print ("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [41]:
token = util.prompt_for_user_token(username,scope,
                           client_id='e67d7b1bc1ee460bbef7484ee3f601ef',
                           client_secret='ba0087b5f4ab4fb8ab88be1850854538',
                           redirect_uri='https://google.com')

In [42]:
spotify = spotipy.Spotify(auth=token)
spotify.current_user_recently_played = types.MethodType(current_user_recently_played, spotify)

In [43]:
dave = spotify.current_user_recently_played(limit=50)
out_file = open("dave.json","w")
out_file.write(json.dumps(dave, sort_keys=True, indent=2))
out_file.close()

In [44]:
Tracks = []

for i in range(50):
   Tracks.append(dave['items'][i]['track']['name'])

TrackID = []

for i in range(50):
   TrackID.append(dave['items'][i]['track']['id'])

Artists = []

for i in range(50):
   Artists.append(dave['items'][i]['track']['artists'][0]['name'])

Albums = []


for i in range(50):
   Albums.append(dave['items'][i]['track']['album']['name'])

for (a, b, c, d) in itertools.zip_longest(TrackID, Tracks, Artists, Albums): 
    print (a, b, c,d )

3YVPpVJvJG2EauXurPA90F Tell It to My Heart Taylor Dayne Tell It to My Heart (Expanded Edition)
7HX9f8AMBl0vQxAoAzLqhS You Make My Dreams - Remastered Daryl Hall & John Oates The Very Best of Daryl Hall / John Oates
0Hw6SCrtU9pFCjgAONpnGZ Uptight (Everything's Alright) Stevie Wonder Up-Tight
2Nz6aF1umHh5Et6I5H581L Hooked on a Feeling Blue Swede Hooked On A Feeling - 40th Anniversary Collection
0Se82Sof9IOUY4VdR9un2A Nobody Knows You When You're Down and Out - Acoustic; Live at MTV Unplugged, Bray Film Studios, Windsor, England, UK, 1/16/1992; 2013 Remaster Eric Clapton Unplugged (Deluxe Edition)
4euYSSvpQZSAQ4gThyL2QD Ain't Wastin' Time No More The Allman Brothers Band Eat A Peach
47bXIv24jthL4vzZy3j7CY Strange Brew Cream Disraeli Gears (Deluxe Edition)
5MtN38MGEWJt60LwtBmFNP Texas Flood Stevie Ray Vaughan Texas Flood (Legacy Edition)
3nktGs7MeEKAq2f7YSpENs Peruvian Cocaine Immortal Technique Revolutionary Vol. 2
6IW81NfsjT7tQ3X5qltEEU White Nigger ILL Bill The Hour Of Reprisal
2iP3AaNo

In [45]:
Test4_df = pd.DataFrame(np.column_stack([TrackID, Tracks, Artists, Albums]), 
                               columns=['Track ID','Tracks', 'Artists', 'Albums'])

Test4_df.shape

(50, 4)

In [46]:
features = []
for i in range(0,len(TrackID),50):
    audio_features = spotify.audio_features(TrackID[i:i+50])
    for track in audio_features:
        features.append(track)
        features[-1]['target'] = 1
features

[{'danceability': 0.638,
  'energy': 0.988,
  'key': 1,
  'loudness': -5.042,
  'mode': 1,
  'speechiness': 0.0921,
  'acousticness': 0.0302,
  'instrumentalness': 1.16e-05,
  'liveness': 0.371,
  'valence': 0.723,
  'tempo': 117.955,
  'type': 'audio_features',
  'id': '3YVPpVJvJG2EauXurPA90F',
  'uri': 'spotify:track:3YVPpVJvJG2EauXurPA90F',
  'track_href': 'https://api.spotify.com/v1/tracks/3YVPpVJvJG2EauXurPA90F',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3YVPpVJvJG2EauXurPA90F',
  'duration_ms': 220880,
  'time_signature': 4,
  'target': 1},
 {'danceability': 0.749,
  'energy': 0.484,
  'key': 5,
  'loudness': -9.944,
  'mode': 1,
  'speechiness': 0.0484,
  'acousticness': 0.13,
  'instrumentalness': 0.137,
  'liveness': 0.0501,
  'valence': 0.939,
  'tempo': 167.084,
  'type': 'audio_features',
  'id': '7HX9f8AMBl0vQxAoAzLqhS',
  'uri': 'spotify:track:7HX9f8AMBl0vQxAoAzLqhS',
  'track_href': 'https://api.spotify.com/v1/tracks/7HX9f8AMBl0vQxAoAzLqhS',
  'analysi

In [47]:
audiofeat_df = pd.DataFrame(features)
audiofeat_df.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'target'],
      dtype='object')

In [48]:
audiofeat_df.drop(audiofeat_df.loc[:,'uri':'target'], inplace = True, axis = 1)

del audiofeat_df['type']

audiofeat_df.head(1)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
0,0.638,0.988,1,-5.042,1,0.0921,0.0302,0.000012,0.371,0.723,117.955,3YVPpVJvJG2EauXurPA90F


In [54]:
Test4 = Test4_df.join(audiofeat_df)

del Test4['id']



In [57]:
Test4.to_csv('dfs/Test4.csv',index=False)

In [58]:
Test1 = pd.read_csv('dfs/Test1.csv')
Test2 = pd.read_csv('dfs/drake.csv')
Test3 = pd.read_csv('dfs/Test3.csv')
Test4 = pd.read_csv('dfs/Test4.csv')

In [59]:
Test2.columns

Index(['Track_ID', 'Tracks', 'album', 'artist', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype='object')

In [60]:
Test2 = Test2.rename(columns = {'Track_ID':'Track ID'})
Test2 = Test2.rename(columns = {'artist':'Artists'})
Test2 = Test2.rename(columns = {'album':'Albums'})

In [61]:
Test1.columns

Index(['Track ID', 'Tracks', 'Artists', 'Albums', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo'],
      dtype='object')

In [62]:
Test2 = Test2[['Track ID', 'Tracks', 'Artists', 'Albums', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo']]

In [63]:
TestSet = pd.DataFrame(columns=['Owner','Track_ID', 'Tracks', 
                                    'Artists', 'Albums', 'danceability', 
                                    'energy','key', 'loudness', 'mode', 
                                    'speechiness', 'acousticness',
                                    'instrumentalness', 'liveness', 
                                    'valence', 'tempo'])

for index, row in Test1.iterrows():
    TestSet.loc[TestSet.shape[0]] =  ['1', row['Track ID'], 
                                         row['Tracks'], row['Artists'],
                                         row['Albums'], row['danceability'], 
                                         row ['energy'],row['key'], 
                                         row['loudness'], row['mode'], 
                                         row['speechiness'],row['acousticness'], 
                                         row['instrumentalness'], row['liveness'],
                                         row['valence'], row['tempo']]

for index, row in Test2.iterrows():
    TestSet.loc[TestSet.shape[0]] =  ['2', row['Track ID'], 
                                         row['Tracks'], row['Artists'],
                                         row['Albums'], row['danceability'], 
                                         row ['energy'],row['key'], 
                                         row['loudness'], row['mode'], 
                                         row['speechiness'],row['acousticness'], 
                                         row['instrumentalness'], row['liveness'],
                                         row['valence'], row['tempo']]
for index, row in Test3.iterrows():
    TestSet.loc[TestSet.shape[0]] =  ['3', row['Track ID'], 
                                         row['Tracks'], row['Artists'],
                                         row['Albums'], row['danceability'], 
                                         row ['energy'],row['key'], 
                                         row['loudness'], row['mode'], 
                                         row['speechiness'],row['acousticness'], 
                                         row['instrumentalness'], row['liveness'],
                                         row['valence'], row['tempo']]
for index, row in Test4.iterrows():
    TestSet.loc[TestSet.shape[0]] =  ['4', row['Track ID'], 
                                         row['Tracks'], row['Artists'],
                                         row['Albums'], row['danceability'], 
                                         row ['energy'],row['key'], 
                                         row['loudness'], row['mode'], 
                                         row['speechiness'],row['acousticness'], 
                                         row['instrumentalness'], row['liveness'],
                                         row['valence'], row['tempo']]
    
TestSet
TestSet.to_csv("dfs/testset.csv",index=False)